<a href="https://colab.research.google.com/github/itsyashkhurana/Big-Data-Analystics/blob/main/Interactive_Google_Maps_and_Charts_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tempfile
import base64
import webbrowser

# HTML content with embedded JavaScript and CSS
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Interactive Map and Charts</title>
    <style>
        /* Basic styles for the map and chart containers */
        #map_container {
            width: 100%;
            height: 500px;
            border: 1px solid #ccc;
            margin-bottom: 20px;
        }
        #chart_container {
            width: 100%;
            height: 400px;
            border: 1px solid #ccc;
        }
        /* Style for info window */
        .custom-info-window {
            font-family: 'Arial', sans-serif;
            padding: 10px;
            max-width: 300px;
        }
        .title {
            font-size: 1.2em;
            font-weight: bold;
            margin-bottom: 5px;
            color: #333;
        }
        .address {
            font-size: 0.9em;
            color: #555;
            margin-bottom: 10px;
        }
        .chart-link {
            color: #007BFF;
            cursor: pointer;
            text-decoration: none;
            font-size: 1em;
        }
        .chart-link:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div id="map_container"></div>
    <div id="chart_container"></div>

    <script>
        // This is a placeholder for your Google Maps API key.  You MUST replace it.
        const apiKey = 'YOUR_API_KEY';

        function initMap() {
            if (!apiKey || apiKey === 'YOUR_API_KEY') {
                document.getElementById('map_container').innerHTML =
                    '<div style="color: red; padding: 10px;">Error: Please replace YOUR_API_KEY with your actual Google Maps API key.</div>';
                return;
            }

            const map = new google.maps.Map(document.getElementById('map_container'), {
                center: { lat: 37.7749, lng: -122.4194 }, // Default: San Francisco
                zoom: 12
            });

            const geocoder = new google.maps.Geocoder();
            const placesService = new google.maps.places.PlacesService(map);

            // Sample data (replace with your actual data)
            const locations = [
                {
                    name: 'Golden Gate Bridge',
                    address: 'Golden Gate Bridge, San Francisco, CA',
                    latitude: 37.8199,
                    longitude: -122.4783,
                    type: 'landmark',
                    data: [
                        ['Year', 'Visitors'],
                        ['2020', 1200000],
                        ['2021', 1500000],
                        ['2022', 1800000],
                        ['2023', 2000000]
                    ]
                },
                {
                    name: 'Alcatraz Island',
                    address: 'Alcatraz Island, San Francisco, CA',
                    latitude: 37.8269,
                    longitude: -122.4229,
                    type: 'historical',
                    data: [
                        ['Year', 'Visitors'],
                        ['2020', 800000],
                        ['2021', 950000],
                        ['2022', 1100000],
                        ['2023', 1250000]
                    ]
                },
                {
                    name: 'Pier 39',
                    address: 'Pier 39, San Francisco, CA',
                    latitude: 37.8090,
                    longitude: -122.4098,
                    type: 'tourist',
                    data: [
                        ['Year', 'Visitors'],
                        ['2020', 1000000],
                        ['2021', 1300000],
                        ['2022', 1600000],
                        ['2023', 1900000]
                    ]
                },
                {
                    name: 'Lombard Street',
                    address: 'Lombard Street, San Francisco, CA',
                    latitude: 37.8022,
                    longitude: -122.4191,
                    type: 'tourist',
                    data: [
                        ['Year', 'Visitors'],
                        ['2020', 700000],
                        ['2021', 850000],
                        ['2022', 900000],
                        ['2023', 1000000]
                    ]
                }
            ];

            // Create markers and info windows
            locations.forEach(location => {
                const marker = new google.maps.Marker({
                    position: { lat: location.latitude, lng: location.longitude },
                    map: map,
                    title: location.name
                });

                const infoWindowContent = document.createElement('div');
                infoWindowContent.className = 'custom-info-window';
                infoWindowContent.innerHTML = `
                    <h2 class="title">${location.name}</h2>
                    <p class="address">${location.address}</p>
                    <button class="chart-link" data-location-name="${location.name}">View Visitor Chart</button>
                `;

                const infowindow = new google.maps.InfoWindow({
                    content: infoWindowContent
                });

                marker.addListener('click', () => {
                    infowindow.open(map, marker);
                });

                // Event listener for the chart link
                infoWindowContent.querySelector('.chart-link').addEventListener('click', () => {
                    drawChart(location.data, location.name);
                });
            });

            // Autocomplete for address search
            const input = document.createElement('input');
            input.id = 'search_input';
            input.type = 'text';
            input.placeholder = 'Enter a location...';
            map.controls[google.maps.ControlPosition.TOP_LEFT].push(input);

            const autocomplete = new google.maps.places.Autocomplete(input);
            autocomplete.bindTo('bounds', map);

            autocomplete.addListener('place_changed', () => {
                const place = autocomplete.getPlace();
                if (!place.geometry) {
                    alert("Place not found!");
                    return;
                }

                map.setCenter(place.geometry.location);
                map.setZoom(15);

                // Clear any previous markers
                map.markers.forEach(marker => marker.setMap(null));
                map.markers = [];

                new google.maps.Marker({
                    position: place.geometry.location,
                    map: map,
                    title: place.name
                });
            });
             map.markers = [];
        }


        // Function to draw the chart using Google Charts
        function drawChart(data, title) {
            google.charts.load('current', { 'packages': ['corechart'] });
            google.charts.setOnLoadCallback(function () {
                const chartData = google.visualization.arrayToDataTable(data);

                const options = {
                    title: title,
                    curveType: 'line',
                    legend: { position: 'bottom' },
                    animation: {
                        duration: 1000,
                        easing: 'out',
                        startup: true
                    }
                };

                const chart = new google.visualization.LineChart(document.getElementById('chart_container'));

                // Clear the chart before drawing a new one
                chart.clearChart = function () {
                    document.getElementById('chart_container').innerHTML = ''; // Clear the div
                };
                chart.clearChart();
                chart.draw(chartData, options);
            });
        }

        // Include the Google Maps script with the API key
        function loadGoogleMaps() {
            if (!apiKey || apiKey === 'YOUR_API_KEY') {
                 document.getElementById('map_container').innerHTML =
                    '<div style="color: red; padding: 10px;">Error: Please replace YOUR_API_KEY with your actual Google Maps API key.</div>';
                 return;
            }
            const script = document.createElement('script');
            script.src = `https://maps.googleapis.com/maps/api/js?key=${apiKey}&libraries=places`;
            script.async = true;
            script.defer = true;
            document.head.appendChild(script);
            script.onload = initMap; // Call initMap after the script is loaded
        }

        // Initialize the map when the page loads
        window.onload = loadGoogleMaps;
    </script>
</body>
</html>
"""

# Save the HTML content to a temporary file
with tempfile.NamedTemporaryFile(delete=False, suffix=".html") as f:
    f.write(html_content.encode('utf-8'))
    html_filename = f.name

# Check if running in an IPython environment (like Colab)
try:
    from IPython.display import IFrame, display
    is_ipython = True
except ImportError:
    is_ipython = False

# Display the HTML in an IFrame if in Colab, otherwise open in a browser
if is_ipython:
    display(IFrame(html_filename, width="100%", height="600"))
else:
    webbrowser.open(html_filename)

In [ ]:
from IPython.display import HTML

HTML('''
<!DOCTYPE html>
<html>
  <head>
      <title>Google Maps + Charts Visualization</title>
          <meta name="viewport" content="initial-scale=1.0">
              <meta charset="utf-8">
                  <style>
                        #map, #chart_div {
                                height: 400px;
                                        width: 100%;
                                                margin-bottom: 20px;
                                                      }
                                                            body {
                                                                    font-family: Arial, sans-serif;
                                                                            margin: 20px;
                                                                                  }
                                                                                      </style>
                                                                                          <!-- Google Maps API -->
                                                                                              <script src="https://maps.googleapis.com/maps/api/js?key=YOUR_API_KEY"></script>
                                                                                                  <!-- Google Charts API -->
                                                                                                      <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
                                                                                                        </head>
                                                                                                          <body>
                                                                                                              <h2>Interactive Google Maps and Charts Visualization</h2>
                                                                                                                  <div id="map"></div>
                                                                                                                      <div id="chart_div"></div>

                                                                                                                          <script type="text/javascript">
                                                                                                                                // Sample Data: City, Latitude, Longitude, Population
                                                                                                                                      const dataPoints = [
                                                                                                                                              ["New York", 40.7128, -74.0060, 8419600],
                                                                                                                                                      ["Los Angeles", 34.0522, -118.2437, 3980400],
                                                                                                                                                              ["Chicago", 41.8781, -87.6298, 2716000],
                                                                                                                                                                      ["Houston", 29.7604, -95.3698, 2328000],
                                                                                                                                                                              ["Phoenix", 33.4484, -112.0740, 1690000]
                                                                                                                                                                                    ];

                                                                                                                                                                                          function initMap() {
                                                                                                                                                                                                  const map = new google.maps.Map(document.getElementById('map'), {
                                                                                                                                                                                                            zoom: 4,
                                                                                                                                                                                                                      center: {lat: 39.8283, lng: -98.5795} // Center of the US
                                                                                                                                                                                                                              });

                                                                                                                                                                                                                                      dataPoints.forEach(([city, lat, lng, population]) => {
                                                                                                                                                                                                                                                const marker = new google.maps.Marker({
                                                                                                                                                                                                                                                            position: {lat, lng},
                                                                                                                                                                                                                                                                        map: map,
                                                                                                                                                                                                                                                                                    title: `${city}: Population ${population}`
                                                                                                                                                                                                                                                                                              });

                                                                                                                                                                                                                                                                                                        const infoWindow = new google.maps.InfoWindow({
                                                                                                                                                                                                                                                                                                                    content: `<strong>${city}</strong><br>Population: ${population.toLocaleString()}`
                                                                                                                                                                                                                                                                                                                              });

                                                                                                                                                                                                                                                                                                                                        marker.addListener('click', () => {
                                                                                                                                                                                                                                                                                                                                                    infoWindow.open(map, marker);
                                                                                                                                                                                                                                                                                                                                                              });
                                                                                                                                                                                                                                                                                                                                                                      });
                                                                                                                                                                                                                                                                                                                                                                            }

                                                                                                                                                                                                                                                                                                                                                                                  google.charts.load('current', {'packages':['corechart']});
                                                                                                                                                                                                                                                                                                                                                                                        google.charts.setOnLoadCallback(drawChart);

                                                                                                                                                                                                                                                                                                                                                                                              function drawChart() {
                                                                                                                                                                                                                                                                                                                                                                                                      const data = google.visualization.arrayToDataTable([
                                                                                                                                                                                                                                                                                                                                                                                                                ['City', 'Population'],
                                                                                                                                                                                                                                                                                                                                                                                                                          ...dataPoints.map(row => [row[0], row[3]])
                                                                                                                                                                                                                                                                                                                                                                                                                                  ]);

                                                                                                                                                                                                                                                                                                                                                                                                                                          const options = {
                                                                                                                                                                                                                                                                                                                                                                                                                                                    title: 'City Populations',
                                                                                                                                                                                                                                                                                                                                                                                                                                                              pieHole: 0.4,
                                                                                                                                                                                                                                                                                                                                                                                                                                                                        legend: { position: 'bottom' }
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                };

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        const chart = new google.visualization.PieChart(document.getElementById('chart_div'));
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                chart.draw(data, options);
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      }

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            window.onload = initMap;
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                </script>
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  </body>
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  </html>
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  ''')


In [ ]:
from IPython.display import HTML

HTML('''
<!DOCTYPE html>
<html>
  <head>
      <title>Google Maps + Charts Visualization</title>
          <meta name="viewport" content="initial-scale=1.0">
              <meta charset="utf-8">
                  <style>
                        #map, #chart_div {
                                height: 400px;
                                        width: 100%;
                                                margin-bottom: 20px;
                                                      }
                                                            body {
                                                                    font-family: Arial, sans-serif;
                                                                            margin: 20px;
                                                                                  }
                                                                                      </style>
                                                                                          <!-- Google Maps API -->
                                                                                              <script src="https://maps.googleapis.com/maps/api/js?key=YOUR_API_KEY"></script>
                                                                                                  <!-- Google Charts API -->
                                                                                                      <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
                                                                                                        </head>
                                                                                                          <body>
                                                                                                              <h2>Interactive Google Maps and Charts Visualization</h2>
                                                                                                                  <div id="map"></div>
                                                                                                                      <div id="chart_div"></div>

                                                                                                                          <script type="text/javascript">
                                                                                                                                // Sample Data: City, Latitude, Longitude, Population
                                                                                                                                      const dataPoints = [
                                                                                                                                              ["New York", 40.7128, -74.0060, 8419600],
                                                                                                                                                      ["Los Angeles", 34.0522, -118.2437, 3980400],
                                                                                                                                                              ["Chicago", 41.8781, -87.6298, 2716000],
                                                                                                                                                                      ["Houston", 29.7604, -95.3698, 2328000],
                                                                                                                                                                              ["Phoenix", 33.4484, -112.0740, 1690000]
                                                                                                                                                                                    ];

                                                                                                                                                                                          function initMap() {
                                                                                                                                                                                                  const map = new google.maps.Map(document.getElementById('map'), {
                                                                                                                                                                                                            zoom: 4,
                                                                                                                                                                                                                      center: {lat: 39.8283, lng: -98.5795} // Center of the US
                                                                                                                                                                                                                              });

                                                                                                                                                                                                                                      dataPoints.forEach(([city, lat, lng, population]) => {
                                                                                                                                                                                                                                                const marker = new google.maps.Marker({
                                                                                                                                                                                                                                                            position: {lat, lng},
                                                                                                                                                                                                                                                                        map: map,
                                                                                                                                                                                                                                                                                    title: `${city}: Population ${population}`
                                                                                                                                                                                                                                                                                              });

                                                                                                                                                                                                                                                                                                        const infoWindow = new google.maps.InfoWindow({
                                                                                                                                                                                                                                                                                                                    content: `<strong>${city}</strong><br>Population: ${population.toLocaleString()}`
                                                                                                                                                                                                                                                                                                                              });

                                                                                                                                                                                                                                                                                                                                        marker.addListener('click', () => {
                                                                                                                                                                                                                                                                                                                                                    infoWindow.open(map, marker);
                                                                                                                                                                                                                                                                                                                                                              });
                                                                                                                                                                                                                                                                                                                                                                      });
                                                                                                                                                                                                                                                                                                                                                                            }

                                                                                                                                                                                                                                                                                                                                                                                  google.charts.load('current', {'packages':['corechart']});
                                                                                                                                                                                                                                                                                                                                                                                        google.charts.setOnLoadCallback(drawChart);

                                                                                                                                                                                                                                                                                                                                                                                              function drawChart() {
                                                                                                                                                                                                                                                                                                                                                                                                      const data = google.visualization.arrayToDataTable([
                                                                                                                                                                                                                                                                                                                                                                                                                ['City', 'Population'],
                                                                                                                                                                                                                                                                                                                                                                                                                          ...dataPoints.map(row => [row[0], row[3]])
                                                                                                                                                                                                                                                                                                                                                                                                                                  ]);

                                                                                                                                                                                                                                                                                                                                                                                                                                          const options = {
                                                                                                                                                                                                                                                                                                                                                                                                                                                    title: 'City Populations',
                                                                                                                                                                                                                                                                                                                                                                                                                                                              pieHole: 0.4,
                                                                                                                                                                                                                                                                                                                                                                                                                                                                        legend: { position: 'bottom' }
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                };

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        const chart = new google.visualization.PieChart(document.getElementById('chart_div'));
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                chart.draw(data, options);
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      }

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            window.onload = initMap;
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                </script>
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  </body>
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  </html>
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  ''')
